#**Text classification with TensorFlow Hub: Movie reviews**

####**Firstly import the necessary libraries and check the libraries**

In [0]:
import tensorflow as tf
if (tf.config.experimental.list_physical_devices("GPU") == [] or tf.__version__ != '2.0.0'):
  !pip install tensorflow-gpu
  print("** Tensorflow updated **")
import numpy as np
!pip install -q tensorflow-hub
!pip install -q tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
print("Version : ",tf.__version__)
print("Eager mode : ",tf.executing_eagerly())
print("Hub Version : ",hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version :  1.15.0
Eager mode :  False
Hub Version :  0.7.0
GPU is available


####**Download the IMDB dataset**

* Split the training set into 60% and 40%, so we'll end up with 15,000 examples
 for training, 10,000 examples for validation and 25,000 examples for testing.

In [3]:
trainValidationSplit = tfds.Split.TRAIN.subsplit([6,4])
(trainData, validationData), testData = tfds.load(
    name='imdb_reviews',
    split=(trainValidationSplit, tfds.Split.TEST),
    as_supervised = True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


#**Explore the data**
* Let's take a moment to understand the format of the data. 
* Each example is a sentence representing the movie review and a corresponding label. 
* The sentence is not preprocessed in any way. 
* The label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review.

Let's print first 10 examples.

In [0]:
trainExamplesBatch, trainLabelsBatch = next(iter(trainData.batch(10)))
trainExamplesBatch

<tf.Tensor: id=12711, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story


####Let's print the top ten labels

In [0]:
trainLabelsBatch

<tf.Tensor: id=12712, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

#**Build the model**

* In this example, the input data consists of sentences. 
The labels to predict are either 0 or 1.


> One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have three advantages :
1. We don't have to worry about text preprocessing
2. We can benefit from transfer learning
3. The embedding has a fixed size, so it's simpler to process

For this example we will use a pre-trained text embedding model from TensorFlow Hub called " https://google/tf2-preview/gnews-swivel-20dim/1 "

Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. Note that no matter the length of the input text, the output shape of the embeddings is: **(num_examples, embedding_dimension).**




In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hubLayer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)
hubLayer(trainExamplesBatch[:3])

<tf.Tensor: id=12894, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

####Now let's create the full model

In [0]:
model = tf.keras.Sequential()
model.add(hubLayer)                                         # https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))   # sigmoid : binary classification 
model.summary()        

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_6 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


###Loss function and optimizer

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # mean_squared_error but not better than this
              metrics=['accuracy'])

#**Train the model**
* Train the model for 20 epochs in mini-batches of 512 samples.
* During training, monitor the loss and accuracy of the model in 10,000 samples from the validation set

In [0]:
history = model.fit(trainData.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data= validationData.batch(512),
                    verbose=1)


Epoch 1/20
30/30 [==============================] - 9s 285ms/step - loss: 0.9400 - accuracy: 0.5053 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 8s 262ms/step - loss: 0.7264 - accuracy: 0.5325 - val_loss: 0.6831 - val_accuracy: 0.5753
Epoch 3/20
30/30 [==============================] - 9s 286ms/step - loss: 0.6649 - accuracy: 0.6133 - val_loss: 0.6326 - val_accuracy: 0.6516
Epoch 4/20
30/30 [==============================] - 9s 285ms/step - loss: 0.6141 - accuracy: 0.6728 - val_loss: 0.5937 - val_accuracy: 0.6923
Epoch 5/20
30/30 [==============================] - 8s 275ms/step - loss: 0.5754 - accuracy: 0.7126 - val_loss: 0.5620 - val_accuracy: 0.7206
Epoch 6/20
30/30 [==============================] - 8s 282ms/step - loss: 0.5442 - accuracy: 0.7401 - val_loss: 0.5352 - val_accuracy: 0.7426
Epoch 7/20
30/30 [==============================] - 9s 289ms/step - loss: 0.5155 - accuracy: 0.7620 - val_loss: 0.5087 - val_accuracy: 0.764

#**Evaluate the model**
And let's see how the model performs. Two values will be returned. Loss and accuracy.

In [0]:
result = model.evaluate(testData.batch(512), verbose=1)
for name, value in zip(model.metrics_names, result):
  print("%s: %.3f" % (name, value))

49/49 [==============================] - 6s 118ms/step - loss: 0.3187 - accuracy: 0.8644
loss: 0.319
accuracy: 0.864


#**The End**
>**See you next coding**